<a href="https://colab.research.google.com/github/darkzard05/graph_neural_network/blob/main/APPNP_by_pytorch_geometric(Cora_dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch_geometric

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 28.4 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 35.0 MB/s 
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 750 kB 38.2 MB/s 
     |████████████████████████████████| 407 kB 26.0 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=489ab363a3fb566c770c997090508815c5710793e2ecb99c20acfed84bcdc128
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import APPNP

print(torch.__version__)
print(torch.version.cuda)

1.10.0+cu111
11.1


In [ ]:
dataset_name = 'Cora'
dataset = Planetoid(root='/tmp/'+dataset_name, name=dataset_name)

Processing...
Done!


In [ ]:
data = dataset[0]
print(f'dataset name: {dataset_name}')
print(f'number of nodes: {data.num_nodes}, number of edges: {data.num_edges}')
print(f'number of classes: {dataset.num_classes}, number of edge features: {data.num_edge_features}')
print(data.train_mask[:20])
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

dataset name: Cora
number of nodes: 2708, number of edges: 10556
number of classes: 7, number of edge features: 0
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])
tensor(140) tensor(500) tensor(1000)


In [ ]:
class Model(torch.nn.Module):
    def __init__(self, hidden, K, alpha):
        super().__init__()
        # self.layer1 = ARMAConv(dataset.num_features, hidden, dropout=0.75)
        # self.layer2 = ARMAConv(hidden, dataset.num_classes, dropout=0.75)
        self.layer1 = Linear(dataset.num_features, hidden)
        self.layer2 = Linear(hidden, dataset.num_classes)
        self.prop = APPNP(K=K, alpha=alpha)

    def reset_parameters(self):
        self.layer1.reset_parameters()
        self.layer2.reset_parameters()

    def forward(self, data):
        x, edge_index  = data.x, data.edge_index
        x = F.dropout(x, training=self.training)
        x = self.layer1(x).relu()
        x = F.dropout(x, training=self.training)
        x = self.layer2(x).relu()
        x = self.prop(x, edge_index)
        return x

In [ ]:
print('cuda' if torch.cuda.is_available() else 'cpu', 'is available')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(hidden=128, K=50, alpha=0.1).to(device)
print(model)
model.reset_parameters()
print({i:list(j.shape) for i, j in model.named_parameters()})

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-3)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    loss = criterion(pred[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(mask):
    model.eval()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    correct = pred.argmax(dim=1)[mask] == data.y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc

best_acc = 0
epochs = 1000
for epoch in range(1, epochs+1):
    loss = train()
    train_acc = test(data.train_mask)
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    if best_acc < test_acc:
        best_acc = test_acc
        print(f'epoch: {epoch} | loss: {loss.item():.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc} | test_acc: {test_acc}')

cuda is available
Model(
  (layer1): Linear(in_features=1433, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=7, bias=True)
  (prop): APPNP(K=50, alpha=0.1)
)
{'layer1.weight': [128, 1433], 'layer1.bias': [128], 'layer2.weight': [7, 128], 'layer2.bias': [7]}
epoch: 1 | loss: 1.9484 | train_acc: 0.6286 | val_acc: 0.586 | test_acc: 0.61
epoch: 3 | loss: 1.8107 | train_acc: 0.8571 | val_acc: 0.66 | test_acc: 0.69
epoch: 4 | loss: 1.7003 | train_acc: 0.8929 | val_acc: 0.698 | test_acc: 0.734
epoch: 5 | loss: 1.5584 | train_acc: 0.9571 | val_acc: 0.734 | test_acc: 0.771
epoch: 6 | loss: 1.3885 | train_acc: 0.9643 | val_acc: 0.762 | test_acc: 0.795
epoch: 7 | loss: 1.2140 | train_acc: 0.9643 | val_acc: 0.78 | test_acc: 0.808
epoch: 11 | loss: 0.6086 | train_acc: 0.9571 | val_acc: 0.802 | test_acc: 0.815
epoch: 12 | loss: 0.5563 | train_acc: 0.9643 | val_acc: 0.8 | test_acc: 0.836
epoch: 13 | loss: 0.4273 | train_acc: 0.9786 | val_acc: 0.804 | test_acc: 0.838
epo